# Harmonization across studies with PIC-SURE

This tutorial notebook will demonstrate how to query and work with the BioData Catalyst studies, particularly cross-study harmonization. For a more step-by-step introduction to the python PIC-SURE API, see the `1_PICSURE_API_101.ipynb` notebook.

**Before running this notebook, please be sure to review the \"Get your security token\" documentation, which exists in the NHLBI_BioData_Catalyst [README.md file](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/tree/master/NHLBI_BioData_Catalyst#get-your-security-token). It explains about how to get a security token, which is mandatory to access the databases.**

-----

# Environment set-up

### System Requirements
R >= 3.4

### Install packages

In [ ]:
system(command = 'conda install -c conda-forge r-devtools --yes')
devtools::install_github("hms-dbmi/pic-sure-r-client", ref="master", force=T, quiet=TRUE)
devtools::install_github("hms-dbmi/pic-sure-r-adapter-hpds", ref="master", force=T, quiet=TRUE)
devtools::install_github("hms-dbmi/pic-sure-biodatacatalyst-r-adapter-hpds", ref="new-search", force=T)
library(hpds)


In [ ]:
install.packages('dplyr')
library(dplyr)

## Connecting to a PIC-SURE Network

In [ ]:
# Uncomment production URL when testing in production
# PICSURE_network_URL = "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure"
PICSURE_network_URL = "https://biodatacatalyst.integration.hms.harvard.edu/picsure"
token_file <- "token.txt"
token <- scan(token_file, what = "character")
connection <- picsure::connect(PICSURE_network_URL, token)
authPicSure = bdc::use.authPicSure(connection)

-----

## Harmonizing variables with PIC-SURE
One of the key challenges to conducting analyses with several studies is ensuring correct data harmonization, or combining of data from different sources. There are many harmonization techniques, but this notebook will demonstrate an approach to finding and extracting similar variables from different studies in PIC-SURE. Two examples of this will be shown:
1. Retrieving variables for *sex and gender* across studies with BMI
2. Harmonizing the *"orthopnea"* and *"pneumonia"* variables across studies


*For more information about the TOPMed DCC Harmonized Data Set in PIC-SURE, please refer to the [`2_TOPMed_DCC_Harmonized_Variables_analysis.ipynb`](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/blob/master/NHLBI_BioData_Catalyst/python/2_TOPMed_DCC_Harmonized_Variables_analysis.ipynb) notebook*

-----

## Sex and gender variables across studies
<font color='darkgreen'>**Goal: Create harmonized variables for sex and BMI which combine data from multiple studies**</font> 

These variables are labelled differently for each of these studies. For example, some use the keyword `sex` while others use `gender`. To acccount for these differences, we need to develop a way to search for multiple keywords at once.

Let's start by searching for `sex` and `gender` to gain a better understanding of the variables that exist in PIC-SURE with these terms. The `bdc::find.in.dictionary()` function can take in regular expressions. Here, we take advantage of this by searching for both sex and gender in the same regular expression.


In [ ]:
dictionary <- bdc::use.dictionary(connection) # set up the variable dictionary
sex_dictionary <- bdc::find.in.dictionary(dictionary, "sex|gender")
sex_df <- bdc::extract.dataframe(sex_dictionary)
head(sex_df, 3)

After reviewing the variables using the dataframe (or the [user interface](https://picsure.biodatacatalyst.nhlbi.nih.gov/psamaui/login)), let's say we are interested in sex/gender variables from the following studies:
- ECLIPSE (Evaluation of COPD Longitudinally to Identify Predictive Surrogate Endpoints) (phs001252)
- EOCOPD (Early Onset of COPD) (phs000946)

We can find the study IDs in the Data Access Dashboard in the user interface.

We have already used the PIC-SURE API dictionary object and find method to find variables which contain the keywords sex and gender. Let's filter these results to our desired studies of interest.

In [ ]:
# which sex / gender variables are part of ECLIPSE?
# create a subset of only ECLIPSE sex/gender vars
eclipse_sex_df <- sex_df %>%
    filter(grepl('phs001252', study_id)) %>%
    select(var_name, var_description, values, HPDS_PATH)

eclipse_sex_df

Many studies may have multiple variables with similar names and descriptions that come from different data tables. This will vary from study to study based on how the original study was conducted and organized. 

We will approach this complication by examining the data associated with each sex / gender variables in ECLIPSE and determining which would be the best fit for our analysis.

Because we want to examine the patient level data, we will create a query using all sex/gender variables in ECLIPSE. We will add these variables to our query using the `anyof` method, as we are interested in all observations with a value for any of our chosen variables.

In [ ]:
eclipse_sex_query <- bdc::new.query(authPicSure) # Start a new query
invisible(lapply(eclipse_sex_df$HPDS_PATH, bdc::query.select.add, query = eclipse_sex_query))
eclipse_sex_results <- bdc::query.run(eclipse_sex_query)
head(eclipse_sex_results)

By previewing the resulting dataframe, we can see that not all of the sex and gender variables are complete for our participant subset. 

Let's see which sex/gender variables are the most complete for our dataset by counting the number of NA values in each one. 

In [ ]:
# first let's see which sex/gender variables are the most complete for our dataset by counting the number of NA values
as.data.frame(colSums(eclipse_sex_results != '')) 

We can see that there are a few variables with 0 NA values, we would like to focus on these.

In [ ]:
# let's focus on those variables which do not have any NA values
filtered_eclipse_sex_results = eclipse_sex_results.loc[:, eclipse_sex_results.isna().sum() == 0]
filtered_eclipse_sex_results